In [3]:
import torch
from torch import nn
from pathlib import Path
from datetime import datetime

class LeaflictionCNN(nn.Module):

    def __init__(self, num_classes):
        """
        CNN architecture
        Params:
        - num_classes: Number of classes to predict.
        """
        super().__init__()
        def CBR(in_c, out_c):
            return nn.Sequential(
                nn.Conv2d(in_c, out_c, 3, padding=1, bias=False),
                nn.BatchNorm2d(out_c),
                nn.ReLU(inplace=True),
            )
        self.features = nn.Sequential(
            CBR(3, 16), nn.MaxPool2d(2),
            CBR(16, 32), nn.MaxPool2d(2),
            CBR(32, 64), nn.MaxPool2d(2),
            CBR(64, 128), nn.AdaptiveAvgPool2d(1)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.2),
            nn.Linear(128, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [ ]:
from itertools import count

def make_train_fn():
    counter = count(1)

    def train_one_epoch(model, dataloader, loss_fn, optimizer):
        """Training loop on one epoch, from dataloader"""
        model.train()
        device = next(model.parameters()).device
        epoch = next(counter)

        for batch_idx, (X, y) in enumerate(
            tqdm(dataloader, total=len(dataloader), desc=f"[Epoch {epoch:02d}]"), start=1
            ):
            X, y = X.to(device), y.to(device)

            optimizer.zero_grad()
            logits = model(X)
            loss = loss_fn(logits, y)
            loss.backward()
            optimizer.step()
    
    return train_one_epoch


In [ ]:
import numpy as np
from tqdm import tqdm
from typing import Any

def evaluate(model, dataloader, split, loss_fn=None) -> dict[str, Any]:
    """
    Model evaluation on dataloader
    Returns dict with accuracy, loss (if provided), y_true, y_pred.
    """
    model.eval()
    device = next(model.parameters()).device
    y_true, y_pred = [], []
    total_loss = 0.0
    tqdm_desc = f"Evaluation {split:>5} [{len(dataloader):>3} batches | {len(dataloader.dataset):>5} imgs]"

    with torch.no_grad():
        pbar = tqdm(dataloader, total=len(dataloader), desc=tqdm_desc)
        for X, y in pbar:
            X, y = X.to(device), y.to(device)
            logits = model(X)
            preds = logits.argmax(1)

            y_true.extend(y.tolist())
            y_pred.extend(preds.cpu().tolist())

            if loss_fn is not None:
                total_loss += loss_fn(logits, y).item() * y.size(0)

            n_samples = len(y_true)
            accuracy = np.mean(np.array(y_true) == np.array(y_pred)) if y_true else 0.0
            avg_loss = total_loss / n_samples if (loss_fn and n_samples) else None

            postfix = {"accuracy": f"{accuracy:.4f}"}
            if avg_loss is not None:
                postfix["loss"] = f"{avg_loss:.4f}"
            pbar.set_postfix(postfix)

    return {
        "accuracy": accuracy,
        "loss": avg_loss,
        "y_true": y_true,
        "y_pred": y_pred,
    }


In [108]:
import sys
import os

sys.path.append(os.path.abspath(".."))

from scripts.utils.data import LeaflictionData

batch_size = 64

data = LeaflictionData(
    original_dir="/Users/a.villa.massone/Code/42/42_Leaffliction/images/Apple",
    force_preproc=False,
    test_split=0.2,
    val_split=0.2,
    batch_size=batch_size,
)

class_names = list(data.class_to_idx.keys())
labels = list(data.class_to_idx.values())

meta = {}
meta['class_names'] = class_names
meta['labels'] = labels
meta['train_size'] = len(data.loaders['train'].dataset)
meta['val_size'] = len(data.loaders['val'].dataset)
meta['test_size'] = len(data.loaders['test'].dataset)
meta['batch_size'] = data.loaders['train'].batch_size
meta['train_batches'] = len(data.loaders['train'])
meta['val_batches'] = len(data.loaders['val'])
meta['test_batches'] = len(data.loaders['test'])
for k, v in meta.items():
    print(f"{k:<15}{v}")


[2025-09-18 16:12:55] [INFO] scripts.utils.data: Initializing LeaflictionData
[2025-09-18 16:12:55] [INFO] scripts.utils.data: Datasets are ready. Skipping preprocessing
[2025-09-18 16:12:55] [INFO] scripts.utils.data: Successfully loaded '/Users/a.villa.massone/Code/42/42_Leaffliction/images_augmented/Apple'
[2025-09-18 16:12:55] [INFO] scripts.utils.data: Successfully loaded '/Users/a.villa.massone/Code/42/42_Leaffliction/images_augmented/Apple_train'
[2025-09-18 16:12:55] [INFO] scripts.utils.data: Successfully loaded '/Users/a.villa.massone/Code/42/42_Leaffliction/images_augmented/Apple_val'
[2025-09-18 16:12:55] [INFO] scripts.utils.data: Successfully loaded '/Users/a.villa.massone/Code/42/42_Leaffliction/images_augmented/Apple_test'
[2025-09-18 16:12:55] [INFO] scripts.utils.data: Successfully loaded '/Users/a.villa.massone/Code/42/42_Leaffliction/images_augmented/Apple_train_raw'
[2025-09-18 16:12:55] [INFO] scripts.utils.data: Data loaders available : dict_keys(['train', 'test'

class_names    ['Apple_Black_rot', 'Apple_healthy', 'Apple_rust', 'Apple_scab']
labels         [0, 1, 2, 3]
train_size     14189
val_size       505
test_size      632
batch_size     64
train_batches  222
val_batches    8
test_batches   10


In [4]:

device = ("cuda" if torch.cuda.is_available()
          else "mps" if torch.backends.mps.is_available()
          else "cpu")
print(f"Using {device} device")

model = LeaflictionCNN(num_classes=4).to(device)
print(model)

Using mps device
LeaflictionCNN(
  (features): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): Sequential(
      (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation

In [6]:
####### TRAINING #############

from torch import nn
import matplotlib.pyplot as plt

learning_rate = 1e-3
weight_decay = 1e-4
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW( model.parameters(), lr=learning_rate, weight_decay=weight_decay)
epochs = 50

best_acc, best_epoch, patience, bad = 0.0, 0, 7, 0
best_state = None
history = {
    "train_loss":[],
    "train_acc":[],
    "val_loss":[],
    "val_acc":[],
}
train_one_epoch = make_train_fn()

print("Starting training of LeaflictionCNN")
for epoch in range(1, epochs + 1):
    # training and evaluations with logs
    train_one_epoch(model=model, dataloader=data.loaders['train'], loss_fn=loss, optimizer=optimizer)
    train_results = evaluate(model=model, dataloader=data.loaders['train'], set="train", loss_fn=loss)
    val_results = evaluate(model=model, dataloader=data.loaders['val'], set="val",  loss_fn=loss)

    # record evaluation history
    history["train_loss"].append(train_results["loss"])
    history["train_acc"].append(train_results["accuracy"])
    history["val_loss"].append(val_results["loss"])
    history["val_acc"].append(val_results["accuracy"])

    # early stopping with patience - restaures best weights
    if val_results['accuracy'] > best_acc:
        best_acc = val_results['accuracy']
        best_epoch = epoch
        best_state = {k: v.cpu() for k,v in model.state_dict().items()}
        bad = 0
    else:
        bad += 1
        if bad >= patience:
            print(f"Early stopping. Restoring state from epoch {best_epoch}")
            model.load_state_dict({k: v.to(device) for k,v in best_state.items()})
            break
        
print("Training done!")


Starting training of LeaflictionCNN
[Epoch 01]


Evaluation: 100%|██████████| 222/222 [00:41<00:00,  5.36it/s]


Evaluation Train: train_loss=0.3245  train_acc=0.8836


Evaluation: 100%|██████████| 8/8 [00:03<00:00,  2.21it/s]


Evaluation Val:   val_loss=0.2548    val_acc=0.9010
[Epoch 02]


...

Evaluation Val:   val_loss=0.0396    val_acc=0.9861
[Epoch 16]


Evaluation: 100%|██████████| 222/222 [00:47<00:00,  4.69it/s]


Evaluation Train: train_loss=0.0288  train_acc=0.9902


Evaluation: 100%|██████████| 8/8 [00:04<00:00,  1.99it/s]

Evaluation Val:   val_loss=0.0262    val_acc=0.9921
Early stopping. Restoring state from epoch 9
Done!


In [62]:
####### EVALUATION #############

print("Starting evaluation of LeaflictionCNN")

test_results = evaluate(model=model, dataloader=data.loaders['test'], split="test")
        
print("Evaluation done!")


Starting evaluation of LeaflictionCNN


Evaluation  test [ 10 batches |   632 imgs]: 100%|██████████| 10/10 [00:04<00:00,  2.20it/s, accuracy=0.9889]

Evaluation done!


In [ ]:
def plot_learning_curves(
    train_loss, 
    val_loss, 
    train_acc, 
    val_acc, 
    out_dir,
    filename="learning_curves.png"
):
    Path(out_dir).mkdir(parents=True, exist_ok=True)
    epochs = range(1, len(train_loss) + 1)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,5))

    # Loss 
    ax1.plot(epochs, train_loss, label="Train Loss")
    ax1.plot(epochs, val_loss,   label="Val Loss")
    ax1.set_xlabel("Epoch")
    ax1.set_ylabel("Loss")
    ax1.set_title("Learning Curve - Loss")
    ax1.legend()
    ax1.grid(True, linestyle="--", alpha=0.4)

    # Accuracy 
    ax2.plot(epochs, train_acc, label="Train Acc")
    ax2.plot(epochs, val_acc,   label="Val Acc")
    ax2.set_xlabel("Epoch")
    ax2.set_ylabel("Accuracy")
    ax2.set_title("Learning Curve - Accuracy")
    ax2.legend()
    ax2.grid(True, linestyle="--", alpha=0.4)

    # plot
    plt.tight_layout()
    out_path = Path(out_dir) / filename
    fig.savefig(out_path, bbox_inches="tight", dpi=150)
    plt.close(fig)

    print(f"Saved learning curves plots to: \n- {out_path}")


In [ ]:
import csv

def save_history_csv(history, out_dir):
    """save training history as csv"""
    filename = "learning_history.csv"
    out_path = Path(out_dir) / Path(filename)
    Path(out_path).parent.mkdir(parents=True, exist_ok=True)
    
    with open(out_path, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["epoch", "train_loss", "train_acc", "val_loss", "val_acc"])
        for i in range(len(history["train_loss"])):
            writer.writerow([i+1,
                             history["train_loss"][i],
                             history["train_acc"][i],
                             history["val_loss"][i],
                             history["val_acc"][i]])
    print(f"Saved history to: \n- {out_path}")

In [106]:
import pandas as pd
from sklearn.metrics import classification_report


def export_classification_reports(
    out_dir,
    class_names,
    train_results=None,
    val_results=None,
    test_results=None,
    base_filename="classification_report",
):
    """
    Exports classification reports for train val and test
    train_results / val_results / test_results : dict avec 'y_true' et 'y_pred'
    """

    Path(out_dir).mkdir(parents=True, exist_ok=True)

    out_txt = Path(out_dir) / f"{base_filename}.txt"
    out_csv = Path(out_dir) / f"{base_filename}.csv"

    if out_txt.exists():
        out_txt.unlink()
    if out_csv.exists():
        out_csv.unlink()

    with out_txt.open("w") as f:
        f.write("===== Classification reports =====\n")
        for k, v in meta.items():
            f.write(f"{k:<15}:{v}\n")
        f.write("\n\n")

    def _process_split(results, split):
        """Append exportation for one split"""
        if results is None:
            return
        y_true = np.asarray(results["y_true"])
        y_pred = np.asarray(results["y_pred"])
        labels = np.unique(y_true)

        # text
        cr_txt = classification_report(
            y_true, y_pred, labels=labels, target_names=class_names,
            digits=3, zero_division=0
        )

        if out_csv.exists():
            out_csv.unlink()
        with out_txt.open("a") as f:
            f.write(f"===== {split.upper()} =====\n")
            f.write(cr_txt + "\n\n")

        # csv
        cr_dict = classification_report(
            y_true, y_pred, labels=labels, target_names=class_names,
            digits=3, zero_division=0, output_dict=True
        )
        df = pd.DataFrame(cr_dict).T
        idx_map = {str(lbl): name for lbl, name in zip(labels, class_names)}
        df = df.rename(index=idx_map)
        df["split"] = split

        if out_csv.exists():
            df.to_csv(out_csv, mode="a", header=False)
        else:
            df.to_csv(out_csv, index=True)

        print(f"Saved classification report for '{split}' to:\n- {out_txt}\n- {out_csv}")


    for split, results in {
        "train": train_results,
        "val": val_results,
        "test": test_results,
    }.items():
        _process_split(results, split)


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

def export_confusion_matrices(
    train_results=None,
    val_results=None,
    test_results=None,
    out_dir=".",
    class_names=None,
    filename="confusion_matrices.png",
):
    """Export confusion matrices in a png"""

    def _plot_confusion_matrix(ax, y_true, y_pred, labels, class_names, title):
        """Helper pour tracer une matrice de confusion sur un axe donné."""
        cm = confusion_matrix(y_true, y_pred, labels=labels)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
        disp.plot(ax=ax)
        ax.set_title(title)

    available = {
        "Train": train_results,
        "Validation": val_results,
        "Test": test_results,
    }
    splits = {k: v for k, v in available.items() if v is not None}

    if not splits:
        raise ValueError("No split provided")

    labels = range(len(class_names))

    # Plot
    if len(splits) == 3:
        fig, axes = plt.subplots(2, 2, figsize=(12, 10))
        axes = axes.flatten()
        fig.delaxes(axes[-1])

    else:
        fig, axes = plt.subplots(1, len(splits), figsize=(5 * len(splits), 5))
        axes = np.atleast_1d(axes)

    mapping = dict(zip(splits.keys(), axes))

    for split, results in splits.items():
        y_true = np.asarray(results["y_true"])
        y_pred = np.asarray(results["y_pred"])
        _plot_confusion_matrix(
            mapping[split], y_true, y_pred, labels, class_names, split
        )
    for ax in mapping.values():
        ax.tick_params(axis="x", labelrotation=45, labelsize=8)
    fig.suptitle("Confusion Matrices", y=0.98)
    fig.tight_layout()

    # Save
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)
    out_path = out_dir / filename
    fig.savefig(out_path, dpi=150, bbox_inches="tight")
    plt.close(fig)

    print(f"Saved confusion matrices to:\n- {out_path}")


In [ ]:
from torchinfo import summary

def export_model_architecture(
    model, 
    dataloader, 
    out_dir,
    filename="model_architecture.txt"):
    """
    Save model architecture and a detailed summary into a text file.
    """

    X, _ = next(iter(dataloader))
    input_size = tuple(X.shape)
    device = next(model.parameters()).device

    plain_arch = str(model)
    model_summary = summary(model, input_size=input_size, device=device, verbose=0)

    text = (
        "===== MODEL ARCHITECTURE (str(model)) =====\n"
        f"{plain_arch}\n\n"
        "===== DETAILED SUMMARY (torchinfo) =====\n"
        f"{model_summary}\n"
    )

    out_path = Path(out_dir) / filename
    out_path.parent.mkdir(parents=True, exist_ok=True)

    out_path.write_text(text)

    print(f"Model architecture + summary saved to :\n-{out_path}")


In [80]:
out_dir = "LeaflictionCNN_" + str(datetime.now().strftime("%Y-%m-%d %H:%M"))

In [109]:

export_model_architecture(
    model=model, 
    dataloader=data.loaders['train'], 
    out_dir=out_dir
    )
plot_learning_curves(
    train_loss = history['train_loss'], 
    train_acc = history['train_acc'], 
    val_loss = history['val_loss'], 
    val_acc = history['val_acc'], 
    out_dir=out_dir
    )

save_history_csv(history=history, out_dir=out_dir)

export_confusion_matrices(
    train_results=train_results, 
    val_results=val_results, 
    test_results=test_results,
    out_dir=out_dir, 
    class_names=class_names
    )

export_classification_reports(
    train_results=train_results, 
    val_results=val_results, 
    test_results=test_results, 
    out_dir=out_dir, 
    class_names=class_names
    )


Model architecture + summary saved to :
-LeaflictionCNN_2025-09-18 15:08/model_architecture.txt
Saved learning curves plots to: 
- LeaflictionCNN_2025-09-18 15:08/learning_curves.png
Saved history to: 
- LeaflictionCNN_2025-09-18 15:08/learning_history.csv
Saved confusion matrices to:
- LeaflictionCNN_2025-09-18 15:08/confusion_matrices.png
Saved classification report for 'train' to:
- LeaflictionCNN_2025-09-18 15:08/classification_report.txt
- LeaflictionCNN_2025-09-18 15:08/classification_report.csv
Saved classification report for 'val' to:
- LeaflictionCNN_2025-09-18 15:08/classification_report.txt
- LeaflictionCNN_2025-09-18 15:08/classification_report.csv
Saved classification report for 'test' to:
- LeaflictionCNN_2025-09-18 15:08/classification_report.txt
- LeaflictionCNN_2025-09-18 15:08/classification_report.csv
